In [1]:
from transformers import pipeline
import json
import os
key=os.getenv("HUGGINGFACE_API_KEY")
emotion_model = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", token=key)

while True:
    query=input()
    if query=="exit": break
    print(query, json.dumps(emotion_model(query, top_k=28)))

2025-02-16 22:06:03.908758: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-16 22:06:04.773596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-16 22:06:04.773732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-16 22:06:04.892757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-16 22:06:05.146782: I tensorflow/core/platform/cpu_feature_guar

I am feeling depressed because of my mom [{"label": "sadness", "score": 0.804105818271637}, {"label": "disappointment", "score": 0.2157609462738037}, {"label": "neutral", "score": 0.02382960543036461}, {"label": "annoyance", "score": 0.021625010296702385}, {"label": "realization", "score": 0.012930833734571934}, {"label": "grief", "score": 0.012079876847565174}, {"label": "disapproval", "score": 0.00969152431935072}, {"label": "nervousness", "score": 0.00935348216444254}, {"label": "approval", "score": 0.009168700315058231}, {"label": "remorse", "score": 0.008994612842798233}, {"label": "joy", "score": 0.007684770505875349}, {"label": "love", "score": 0.005724872462451458}, {"label": "anger", "score": 0.005535678938031197}, {"label": "caring", "score": 0.004993458744138479}, {"label": "admiration", "score": 0.004297302104532719}, {"label": "amusement", "score": 0.0038930100854486227}, {"label": "fear", "score": 0.0031285672448575497}, {"label": "disgust", "score": 0.002900524064898491}

In [3]:
ans=emotion_model("I am happy", top_k=28)   
for i in range(28):
    print(ans[i]["label"], end=", ")

joy, neutral, approval, relief, admiration, excitement, gratitude, caring, amusement, love, optimism, realization, annoyance, disapproval, sadness, pride, confusion, anger, desire, disappointment, curiosity, nervousness, surprise, remorse, grief, embarrassment, fear, disgust, 

In [4]:
emotion_model("You're asking how I am, honestly I don't even know anymore", top_k=3)

[{'label': 'confusion', 'score': 0.9218111038208008},
 {'label': 'neutral', 'score': 0.0905587300658226},
 {'label': 'realization', 'score': 0.05267453193664551}]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device="cuda:0"
model = "heegyu/TinyLlama-augesc-context-strategy"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model).eval().to(device)


model.safetensors:  38%|###8      | 839M/2.20G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at heegyu/TinyLlama-augesc-context-strategy and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:

example = """usr: Hi
sys[Question]: Hello, how are you today?
usr: I'm feeling anxious that I am going to lose my job."""

inputs = tokenizer(example, return_tensors="pt").to(device)
logits = model(**inputs).logits.softmax(-1)
print(logits)

label = logits.argmax(-1).item()


ESCONV_STRATEGY = [
    "Question",
    "Restatement or Paraphrasing",
    "Reflection of feelings",
    "Self-disclosure",
    "Affirmation and Reassurance",
    "Providing Suggestions",
    "Information",
    "Others"
]
id2label = {i:k for i, k in enumerate(ESCONV_STRATEGY)}

print(id2label[label])

tensor([[0.1740, 0.0569, 0.0956, 0.2178, 0.1346, 0.1829, 0.0739, 0.0644]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
Self-disclosure


In [7]:
id2label = {i:k for i, k in enumerate(ESCONV_STRATEGY)}
id2label

{0: 'Question',
 1: 'Restatement or Paraphrasing',
 2: 'Reflection of feelings',
 3: 'Self-disclosure',
 4: 'Affirmation and Reassurance',
 5: 'Providing Suggestions',
 6: 'Information',
 7: 'Others'}

In [8]:
from datasets import load_dataset

ds = load_dataset("thu-coai/esconv")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 910
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 195
    })
    test: Dataset({
        features: ['text'],
        num_rows: 195
    })
})

In [ ]:
train_data=ds["train"]["text"]
test_data = (ds["test"]["text"])
validation_data = (ds["validation"]["text"])

# data[:5]

['{"experience_type": "Current Experience", "emotion_type": "anxiety", "problem_type": "job crisis", "situation": "I am on short term disability and I am afraid I will lose my job if I don\'t go back soon.", "survey_score": {"seeker": {"initial_emotion_intensity": "3", "empathy": "5", "relevance": "5", "final_emotion_intensity": "2"}, "supporter": {"relevance": "5"}}, "dialog": [{"text": "Hello good afternoon.", "speaker": "usr"}, {"text": "Hi, good afternoon.", "speaker": "sys", "strategy": "Question"}, {"text": "I\'m feeling anxious that I am going to lose my job.", "speaker": "usr"}, {"text": "Losing a job is always anxious.", "speaker": "sys", "strategy": "Reflection of feelings"}, {"text": "I hope I don\'t.", "speaker": "usr"}, {"text": "Why do you think you will lose your job?", "speaker": "sys", "strategy": "Question"}, {"text": "I am on short term disability and I am not ready to go back to work yet but I do not have any job protection.", "speaker": "usr"}, {"text": "Oh so your

In [11]:
l=["a","b"]
l.extend(["c"])
print(l)

['a', 'b', 'c']


In [ ]:
# #REDO because they are not in usr-sys order always!

# import pandas as pd
# import json

# # test_data = json.loads(test_data)

# # Initialize variables for test_dataFrame construction
# rows = []
# session_id = 1

# # Parse the test_data
# for entry in data:
#     entry=json.loads(entry)
#     problem_type = entry.get("problem_type")
#     emotion_type = entry.get("emotion_type")
#     situation = entry.get("situation")

#     # Iterate over dialog entries in pairs
#     dialog = entry["dialog"]
#     tmp={
#         "text":"",
#         "strategy":[],
#         "speaker":""
#     }
#     for text in dialog:
#         if text["speaker"]=="usr" and tmp["speaker"]!="sys":
#             tmp["speaker"]+=" "+text["text"]
            
#         elif text["speaker"]=="sys" and tmp["speaker"]!="usr":
#             tmp["strategy"].append(text["strategy"])
#         else:
#             rows.append({
#                 "problem": problem_type,
#                 "emot"
#             })
#             tmp=tmp={
#                 "text":"",
#                 "strategy":[],
#                 "speaker":""
#             }
        
            
    
#     session_id += 1

# # Convert to DataFrame
# df = pd.DataFrame(rows)

# # Display the DataFrame
# df.head()


SyntaxError: ':' expected after dictionary key (963566968.py, line 35)

In [15]:
import json
import pandas as pd

# Suppose your raw data is stored in the variable `data`
# where each element is a JSON string (one conversation per line).
# For example:
# data = [
#    '{"experience_type": "Current Experience", "emotion_type": "anxiety", ... }',
#    '{"experience_type": "...", ... }',
#    ...
# ]

training_samples = []  # To store the transformed examples

for line in data:
    # Parse the JSON for one conversation
    conv = json.loads(line)
    dialog = conv["dialog"]

    # First, merge consecutive utterances by the same speaker.
    merged_blocks = []  # Each block is a dict: {speaker, text, [strategies]}
    for utt in dialog:
        speaker = utt["speaker"].strip()
        text = utt["text"].strip()
        strategy = utt.get("strategy")  # might be None for user turns

        if merged_blocks and merged_blocks[-1]["speaker"] == speaker:
            # Same speaker as previous block → merge texts.
            merged_blocks[-1]["text"] += " " + text
            # For system turns, if a strategy is present, add it to the list.
            if speaker == "sys" and strategy:
                merged_blocks[-1]["strategies"].append(strategy)
        else:
            # Start a new block.
            if speaker == "sys":
                # For sys blocks, store the strategy in a list.
                merged_blocks.append({
                    "speaker": speaker,
                    "text": text,
                    "strategies": [strategy] if strategy else []
                })
            else:
                merged_blocks.append({
                    "speaker": speaker,
                    "text": text
                })

    # Next, generate training samples.
    # The idea is: for each system (sys) block (which carries a strategy),
    # create a sample whose input text is the conversation history _before_ that sys block.
    for i, block in enumerate(merged_blocks):
        if block["speaker"] == "sys" and block.get("strategies"):
            # Build the conversation context from all previous blocks.
            context_parts = []
            for b in merged_blocks[:i]:
                if b["speaker"] == "usr":
                    context_parts.append(f"usr: {b['text']}")
                elif b["speaker"] == "sys":
                    # Join the strategies with a comma if there are multiple.
                    strat = ", ".join(b.get("strategies", []))
                    context_parts.append(f"sys({strat}): {b['text']}")
            context = " ".join(context_parts).strip()

            # The label is the strategy (or list of strategies) from the current sys block.
            training_samples.append({
                "text": context,
                "label": block["strategies"]
            })

# Optionally, save the transformed data as a CSV (each row: text, label)
df = pd.DataFrame(training_samples)
df.to_csv("transformed_dataset.csv", index=False)

# For inspection, print the first few samples:
for sample in training_samples[:2]:
    print("Text:", sample["text"])
    print("Label:", sample["label"])
    print("------")


Text: usr: Hello good afternoon.
Label: ['Question']
------
Text: usr: Hello good afternoon. sys(Question): Hi, good afternoon. usr: I'm feeling anxious that I am going to lose my job.
Label: ['Reflection of feelings']
------


In [ ]:
df.shape

(14642, 8)

In [ ]:
df[["session_id", "user_text", "sys_text"]][:20]

,session_id,user_text,sys_text
0,1,Hello good afternoon.,"Hi, good afternoon."
1,1,I'm feeling anxious that I am going to lose my...,Losing a job is always anxious.
2,1,I hope I don't.,Why do you think you will lose your job?
3,1,I am on short term disability and I am not rea...,Oh so your job is not protected and your short...
4,1,I'm afraid that I will lose my job since I'm s...,I see. Have you spoken to HR?
5,1,"I have, but they are telling me that it is up ...",Your department manager is not answering you?
6,1,"I wish I could just call him, but I do not hav...",Have you tried mentioning that to HR?
7,1,"I don't think he has a direct work line, and t...",Yes that is how most employments work about pr...
8,1,Yes! 100%. Every time my phone rings I get ner...,No you should not have to feel you made a mist...
9,1,Thank you for saying that. That does make me f...,You know yourself more than anybody and you ne...


This dataset is very useful has problem, emotion(if finetuning krni ho), and a thing called strategy which is related to empathy and therapeutic convos so we could fit a model for that, there is one but it has B parameters or 2gb, too big for our problem(i think) need a smaller one

Next dataset is AugESC which is aaugmented(ai generated) using ESC dataset although it doesn't have strategy , its only slightly useful and the dataset is not for use other than research tasks but lets see

In [ ]:
from datasets import load_dataset
import pandas as pd
import ast

# Load dataset
dataset = load_dataset("thu-coai/augesc")

# Initialize lists
user_texts = []
sys_texts = []
session_ids = []

# Process each conversation in the dataset
for idx, text in enumerate(dataset["train"]["text"], start=1):
    # Convert string representation to list
    conversation = ast.literal_eval(text)
    
    # Extract messages
    for i in range(0, len(conversation), 2):
        if i + 1 < len(conversation):
            if conversation[i][0] == "usr" and conversation[i+1][0] == "sys":
                user_texts.append(conversation[i][1])
                sys_texts.append(conversation[i+1][1])
                session_ids.append(idx)

# Create DataFrame
df = pd.DataFrame({
    'session_id': session_ids,
    'user_text': user_texts,
    'sys_text': sys_texts
})

print(f"Total conversations: {len(df)}")
print("\nFirst few rows:")
print(df.head())
print(f"\nUnique sessions: {df['session_id'].nunique()}")

Total conversations: 611849

First few rows:
   session_id                                          user_text  \
0           1  My upstairs neighbors make a ton of noise at a...   
1           1  I'm not sure if they're directly in my room, b...   
2           1  They have what sounds like parties, loud music...   
3           1  The only thing I've found that helps a little ...   
4           1  I've complained to my landlord but haven't had...   

                                            sys_text  
0  Ah, I understand, there's more noise in your a...  
1  What sort of noise do you hear? Do you feel th...  
2  I can see how it may be difficult to sleep at ...  
3  Have you tried asking them to turn their music...  
4  I would imagine that they don't care much abou...  

Unique sessions: 65077


In [ ]:
df.head()

,session_id,user_text,sys_text
0,1,My upstairs neighbors make a ton of noise at a...,"Ah, I understand, there's more noise in your a..."
1,1,"I'm not sure if they're directly in my room, b...",What sort of noise do you hear? Do you feel th...
2,1,"They have what sounds like parties, loud music...",I can see how it may be difficult to sleep at ...
3,1,The only thing I've found that helps a little ...,Have you tried asking them to turn their music...
4,1,I've complained to my landlord but haven't had...,I would imagine that they don't care much abou...


In [ ]:
import gdown
import zipfile
import pandas as pd
import os

# Download dataset from Google Drive
url = 'https://drive.google.com/file/d/15KjSeYn3tjiHXiswLgffmxPEoMYtepv2'  # Replace with actual file ID
output = 'SMILE_DATASET_v2.zip'

try:
    # Download if file doesn't exist
    if not os.path.exists(output):
        gdown.download(url, output, quiet=False)
    
    # Extract ZIP
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall('smile_data')
    
    # Load dataset
    df = pd.read_csv('smile_data/SMILE_DATASET_v2.csv')  # Adjust path if different
    
    print("Dataset shape:", df.shape)
    print("\nColumns:", df.columns.tolist())
    print("\nFirst few rows:")
    print(df.head())
    
except Exception as e:
    print(f"Error: {str(e)}")

Error: File is not a zip file


In [ ]:
data=load_dataset("Amod/mental_health_counseling_conversations")
data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [ ]:
df=pd.DataFrame({
    "session_id":range(1, len(data["train"])+1), 
    "user_text": data["train"]["Context"], 
    "sys_text": data["train"]["Response"]
    })

df.head()

,session_id,user_text,sys_text
0,1,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,2,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,3,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,4,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,5,I'm going through some things with my feelings...,I first want to let you know that you are not ...


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
!pip install transformers torch lm-format-enforcer huggingface_hub optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [ ]:
# Requirements if running from Google Colab with a T4 GPU. 

from pydantic import BaseModel, Field
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import pipeline

class AnswerFormat(BaseModel):
    user_emotion: str = Field(..., description="User's emotion while writing the query.", example="A mix of relief, regret and sadness")
    user_problem: str = Field(..., description="The problem/issue user is facing", example="self identity crisis")
    strategy: str = Field(..., description="Strategy to be used to answer user's query", example="A mix of Affirmation and Reassurance and Providing Suggestions")
    output: str = Field(..., description="An output to be sent to the user")
    
# Create a transformers pipeline

hf_pipeline = pipeline('text-generation', model='microsoft/DialoGPT-large')

# Create a character level parser and build a transformers prefix function from it
parser = JsonSchemaParser(AnswerFormat.model_json_schema())
prefix_function = build_transformers_prefix_allowed_tokens_fn(hf_pipeline.tokenizer, parser)



2025-01-07 20:57:33.243157: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 20:57:34.060979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 20:57:34.061066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 20:57:34.205878: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 20:57:34.498488: I tensorflow/core/platform/cpu_feature_guar

: 

In [ ]:
AnswerFormat.model_json_schema()

{'properties': {'user_emotion': {'description': "User's emotion while writing the query.",
   'example': 'A mix of relief, regret and sadness',
   'title': 'User Emotion',
   'type': 'string'},
  'user_problem': {'description': 'The problem/issue user is facing',
   'example': 'self identity crisis',
   'title': 'User Problem',
   'type': 'string'},
  'strategy': {'description': "Strategy to be used to answer user's query",
   'example': 'A mix of Affirmation and Reassurance and Providing Suggestions',
   'title': 'Strategy',
   'type': 'string'},
  'output': {'description': 'An output to be sent to the user',
   'title': 'Output',
   'type': 'string'}},
 'required': ['user_emotion', 'user_problem', 'strategy', 'output'],
 'title': 'AnswerFormat',
 'type': 'object'}

In [ ]:
query= "hi"
# input("Enter the query: ")
prompt = f'''Your task is find out the following things regarding user's query.
User's emotion, 
User's problem(eg. job crisis, loneliness etc), 
A therapy strategy that will help the user the most(eg.
"Questioning to make them open up",
"Restatement or Paraphrasing",
"Reflection of feelings",
"Self-disclosure",
"Affirmation and Reassurance",
"Providing Suggestions"),
Along with this, generate a human-like empathetic message

Provide the result in this schema:
{AnswerFormat.model_json_schema()}

Query: {query}
Answer:\n
'''

# Call the pipeline with the prefix function
output_dict = hf_pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)

# Extract the results
result = output_dict[0]['generated_text'][len(prompt):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


            {"strategy" :


In [ ]:
output_dict

[{'generated_text': 'Your task is find out the following things regarding user\'s query.\nUser\'s emotion, \nUser\'s problem(eg. job crisis, loneliness etc), \nA therapy strategy that will help the user the most(eg.\n"Questioning to make them open up",\n"Restatement or Paraphrasing",\n"Reflection of feelings",\n"Self-disclosure",\n"Affirmation and Reassurance",\n"Providing Suggestions"),\nAlong with this, generate a human-like empathetic message\n\nProvide the result in this schema:\n{\'properties\': {\'user_emotion\': {\'description\': "User\'s emotion while writing the query.", \'example\': \'A mix of relief, regret and sadness\', \'title\': \'User Emotion\', \'type\': \'string\'}, \'user_problem\': {\'description\': \'The problem/issue user is facing\', \'example\': \'self identity crisis\', \'title\': \'User Problem\', \'type\': \'string\'}, \'strategy\': {\'description\': "Strategy to be used to answer user\'s query", \'example\': \'A mix of Affirmation and Reassurance and Providi

In [ ]:
from pydantic import BaseModel, Field
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import build_transformers_prefix_allowed_tokens_fn
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

class AnswerFormat(BaseModel):
    user_emotion: str = Field(..., description="User's emotion while writing the query.", example="A mix of relief, regret and sadness")
    user_problem: str = Field(..., description="The problem/issue user is facing", example="self identity crisis")
    strategy: str = Field(..., description="Strategy to be used to answer user's query", example="A mix of Affirmation and Reassurance and Providing Suggestions")
    output: str = Field(..., description="An output to be sent to the user")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

# Parse the schema and create a prefix function
parser = JsonSchemaParser(AnswerFormat.schema())
prefix_function = build_transformers_prefix_allowed_tokens_fn(tokenizer, parser)

# Create a Hugging Face pipeline for conversational tasks
hf_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    prefix_allowed_tokens_fn=prefix_function
)

# User query
query = "hi"

# Define the schema-based prompt
prompt = f'''Your task is to find out the following things regarding the user's query:
- User's emotion
- User's problem (e.g., job crisis, loneliness, etc.)
- A therapy strategy that will help the user the most (e.g., 
  "Questioning to make them open up",
  "Restatement or Paraphrasing",
  "Reflection of feelings",
  "Self-disclosure",
  "Affirmation and Reassurance",
  "Providing Suggestions")
- A human-like empathetic message

Provide the result in this schema:
{AnswerFormat.model_json_schema()}

Query: {query}
Answer:\n'''

# Generate output using the pipeline
output_dict = hf_pipeline(prompt, max_length=500, num_return_sequences=1)

# Extract the generated result
result = output_dict[0]['generated_text'][len(prompt):]
print(result)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    print(new_user_input_ids)
    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    print(bot_input_ids)
    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    print(chat_history_ids)
    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

2025-01-07 21:40:07.665798: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 21:40:08.450769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 21:40:08.450883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 21:40:08.543681: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 21:40:08.858192: I tensorflow/core/platform/cpu_feature_guar

: 

In [ ]:
import pickle
from transformers import pipeline
import faiss
import time
import numpy as np
from transformers import AutoTokenizer, AutoModel
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import asyncio
import os
import google.generativeai as genai
import torch

2025-01-07 23:45:35.647864: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-07 23:45:36.199938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 23:45:36.200019: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 23:45:36.289418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 23:45:36.471525: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
import GPUtil

def get_gpu_usage():
    """
    Get the current GPU usage as a percentage.

    Returns:
        list: A list of dictionaries containing GPU usage details.
    """
    gpus = GPUtil.getGPUs()  # Get list of GPUs available
    gpu_usage_info = []

    for gpu in gpus:
        gpu_info = {
            'GPU': gpu.name,
            'Memory Total': gpu.memoryTotal,
            'Memory Used': gpu.memoryUsed,
            'Memory Free': gpu.memoryFree,
            'GPU Utilization': gpu.memoryUtil * 100,  # Memory Utilization in percentage
            'GPU Temperature': gpu.temperature  # GPU Temperature (in Celsius)
        }
        gpu_usage_info.append(gpu_info)

    return gpu_usage_info


# 


In [ ]:
class Chatbot:
    def __init__(self):
        print(get_gpu_usage())
        # self.dialo = pickle.load(open(small_model_path, "rb"))
        self.problem_tokenizer = AutoTokenizer.from_pretrained(
            "microsoft/DialoGPT-medium"
        )
        self.problem_model = AutoModel.from_pretrained("microsoft/DialoGPT-small")
        print(get_gpu_usage())
        # self.rag = faiss.read_index(rag_path)
        self.emotion_model = pipeline(
            "text-classification", model="SamLowe/roberta-base-go_emotions"
        )
        print(get_gpu_usage())
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        # self.strategy_tokenizer = AutoTokenizer.from_pretrained(
        #     "heegyu/TinyLlama-augesc-context-strategy"
        # )
        print(get_gpu_usage())
        # self.strategy_model = (
        #     AutoModelForSequenceClassification.from_pretrained(
        #         "heegyu/TinyLlama-augesc-context-strategy"
        #     )
        #     .eval()
        #     .to(self.device)
        # )
        ESCONV_STRATEGY = [
            "Question",
            "Restatement or Paraphrasing",
            "Reflection of feelings",
            "Self-disclosure",
            "Affirmation and Reassurance",
            "Providing Suggestions",
            "Information",
            "Others",
        ]
        self.strategy_id2label = {i: k for i, k in enumerate(ESCONV_STRATEGY)}
        genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
        self.model = genai.GenerativeModel("gemini-1.5-flash-002")
        print(get_gpu_usage())
        print("Initialised\n")


In [ ]:
model = Chatbot()

[{'GPU': 'NVIDIA GeForce RTX 3050 Ti Laptop GPU', 'Memory Total': 4096.0, 'Memory Used': 0.0, 'Memory Free': 3962.0, 'GPU Utilization': 0.0, 'GPU Temperature': 42.0}]
[{'GPU': 'NVIDIA GeForce RTX 3050 Ti Laptop GPU', 'Memory Total': 4096.0, 'Memory Used': 0.0, 'Memory Free': 3962.0, 'GPU Utilization': 0.0, 'GPU Temperature': 42.0}]


Device set to use cuda:0


[{'GPU': 'NVIDIA GeForce RTX 3050 Ti Laptop GPU', 'Memory Total': 4096.0, 'Memory Used': 609.0, 'Memory Free': 3354.0, 'GPU Utilization': 14.8681640625, 'GPU Temperature': 43.0}]
[{'GPU': 'NVIDIA GeForce RTX 3050 Ti Laptop GPU', 'Memory Total': 4096.0, 'Memory Used': 609.0, 'Memory Free': 3354.0, 'GPU Utilization': 14.8681640625, 'GPU Temperature': 43.0}]
[{'GPU': 'NVIDIA GeForce RTX 3050 Ti Laptop GPU', 'Memory Total': 4096.0, 'Memory Used': 609.0, 'Memory Free': 3354.0, 'GPU Utilization': 14.8681640625, 'GPU Temperature': 43.0}]
Initialised

